# Mask creation

#### Note: run inside an environment with numpy==1.20.3 e pylidc==0.2.2

In [22]:
import os
import six
import pickle
import pandas as pd
import SimpleITK as sitk
from functools import reduce
import numpy as np
from skimage.measure import find_contours
import pylidc as pl
from pylidc.utils import consensus
import matplotlib.pyplot as plt

### Criação Máscara

In [25]:
def mask_creation(pid):
    patient_scans = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid)
    scan = patient_scans.first()
    nods = scan.cluster_annotations()
    anns = scan.annotations
    ann = anns[0]
    c = ann.contours[0]
    mask = ann.boolean_mask()
    anns = nods[0]
    
    # Perform concensus consolidation and pad slices for visualization
    cmask, cbbox, masks = consensus(anns, clevel=0.5, pad=[(20,20), (20,20), (0,0)])
    
    return cmask

In [30]:
# Path were the images are stored
input_directory = "/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/manifest-1600709154662/LIDC-IDRI"

# Path to the setup file from radiomics
params_file = "/home/cmonteiro/pyradiomics-master/pyradiomics-master/examples/exampleSettings/Params.yaml"

# List to store the features
masks_list = []

# Ordered list of all the subfolders
patient_folders = sorted(os.listdir(input_directory))

for patient_folder in patient_folders:
    patient_folder_path = os.path.join(input_directory, patient_folder)
    
    # Id of the patient - 'LIDC-IDRI-xxxx'
    patient_id = os.path.basename(patient_folder_path)

    for root, _, files in os.walk(patient_folder_path):
        print(".\n")
        for file in files:
            print("img\n")
            if file.endswith('.dcm'):

                # Creation of the binary mask with SimpleITK
                mask = mask_creation(patient_id) 
                masks_list.append(mask)
                
        
    # Saving of the mask on a folder named patient_id_mask
    output_file = os.path.join(input_directory, patient_id + "_mask") 

    with open(output_file, 'wb') as file:
        pickle.dump(masks_list, file)
    

/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/manifest-1600709154662/LIDC-IDRI/LICENSE_mask
/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/manifest-1600709154662/LIDC-IDRI/LICENSE_mask
/home/cmonteiro/Aulas/Laboratório de IA e CD/Projeto 1/Imagens/manifest-1600709154662/LIDC-IDRI/LIDC-IDRI-0068_mask
.

.

.

img

img

img



KeyboardInterrupt: 